
# Tyre strategies during a race

Plot all drivers' tyre strategies during a race.


In [ ]:
import fastf1
import fastf1.plotting
from matplotlib import pyplot as plt

Load the race session



In [ ]:
session = fastf1.get_session(2023,9,'R')
session.load()
compound_color = {'HARD': '#f0f0ec', 'INTERMEDIATE': '#43b02a', 'MEDIUM': '#ffd12e', 'SOFT': '#da291c', 'TEST-UNKNOWN': '#434649', 'UNKNOWN': '#00ffff', 'WET': '#0067ad'}
laps = session.laps

Get the list of driver numbers



In [ ]:
drivers = session.drivers
print(drivers)

Convert the driver numbers to three letter abbreviations



In [ ]:
drivers = [session.get_driver(driver)["Abbreviation"] for driver in drivers]
print(drivers)

We need to find the stint length and compound used
for every stint by every driver.
We do this by first grouping the laps by the driver,
the stint number, and the compound.
And then counting the number of laps in each group.



In [ ]:
stints = laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.rename(columns={"LapNumber": "StintLength"})
laps[["Driver", "Stint", "Compound", "LapNumber"]]

The number in the LapNumber column now stands for the number of observations
in that group aka the stint length.



Now we can plot the strategies for each driver



In [ ]:
driver_stints = stints.loc[stints["Driver"] == 'VER']
driver_stints

In [ ]:
driver_stints.iterrows()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for driver in drivers:
    driver_stints = stints.loc[stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=compound_color[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]